# 本章简介
另一种常用于降维方法就是特征抽取，特征抽取可以将原始数据集变化到一个维度更低的新的特征子空间，从而对数据进行压缩，避免维度灾难  
本章读者将学习三种降维技术：  
+ 无监督数据压缩——主成分分析(PCA)
+ 基于类别可分最大化的监督降维技术——线性判别分析(LDA)
+ 通过核主成分分析进行非线性降维 5  

# 5.1 无监督数据降维技术-主成分分析
> 本章介绍了主成分分析(PCA)的原理、实现和使用方法

### 问题：特征选择和特征抽取有什么区别？
> 特征抽取后的新特征是原来特征的一个映射，而特征选择后的特征是原来特征的一个子集 5  

特征抽取算法会将数据转换或者映射到一个新的特征空间，可以理解为：在尽可能保持相关信息的情况下，对数据进行压缩的一种方法  
特征抽取通常用于提高计算效率，同样也可以帮助我们降低"维度灾难"——尤其当模型不适于正则化处理时  
主成分分析(PCA)是一种广泛应用于不同领域的无监督线性数据转换技术，其突出作用是降维  
简而言之，PCA的目标是在高维数据中找到最大方差的方向，并将数据映射到一个维度不大于原始数据的新的子空间上 5  
如下图所示，新的子空间上正交的坐标轴（主成分）可被解释为方差最大的方向。在此，$x_1$和$x_2$为原始特征的坐标轴，而PC1和PC2即为主成分  
![5-1](../syn_pic/py_machine_learning/5-1.png)
如果使用PCA降维，我们将构建一个$d\times k$维的转换矩阵W，这样就可以将一个样本向量x映射到一个新的k维特征子空间上去，此空间的维度小于原始的d维特征空间：  
$$x=[x_1,x_2,\dots,x_d]，x\in{R}^d$$
$$\downarrow xW,W\in{R}^{d\times k}$$
$$z=[z_1,z_2,\dots,z_k],z\in{R}^k$$
完成从原始d维数据到新的k维子空间(一般情况下$k\ll d$)的转换后，第一主成分的方差应该是最大的，由于各主成分之间是不相关的（正交的），后续各主成分也具备尽可能大的方差  
需要注意的是，主成分的方向对数据值的范围高度敏感，如果特征的值使用不同的度量标准，我们需要先对特征进行标准化处理  
我们先通过以下几个步骤来概括以下算法的流程：
1. 对原始d维数据集做标准化处理 5\*2    
2. 构造样本的协方差矩阵
3. 计算协方差矩阵的特征值和相应的特征向量
4. 选择与前k个最大特征值对应的特征向量，其中k为新特征空间的维度($k\le d$)
5. 通过前k个特征向量构建映射矩阵W
6. 通过映射矩阵W将d维的输入数据集X转换到新的k维特征子空间 5  

### 问题：如何理解主成分分析降维的原理？
> 主成分简单来说就是对结果影响最大的成分，而成分包含在各个特征中  
特征抽取的含义就是把成分从特征中抽取出来，将成分重新组合后映射到新的k维特征空间中  
这种抽取并组合的原理是通过计算贡献方差来找出主成分，并通过映射矩阵将数据转换到k维特征空间  
其中贡献方差是基于协方差矩阵转换成特征对中的特征值计算出来的，而映射矩阵是由特征对中的特征向量构造的 5  
  

## 5.1.1 总体方差与贡献方差
> 本节介绍了如何通过协方差矩阵转换成特征对（特征值和特征向量），并使用特征值计算贡献方差的方法

本节，我们将学习主成分分析算法的前四个步骤：数据标准化、构造协方差矩阵、获得协方差矩阵的特征值和特征向量，以及按降序排列特征值所对应的特征向量  

In [ ]:
import pandas as pd
'''
python	pandas	dataframe	to_csv() index

df_winecsv = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',
                      header=None)
df_winecsv.columns = ['Class label', 'Alcohol', 
                   'Malic acid', 'Ash',
                  'Alcalinity of ash', 'Magnesium',
                  'Total phenols', 'Flavanoids',
                  'Nonflavanoid phenols',
                  'Proanthocyanins',
                  'Color intensity', 'Hue',
                  'OD280/OD315 of diluted wines',
                  'Proline']
df_winecsv.to_csv('../syc_data/py_machine_learning/wine.data', index=False)
df_winecsv.head()
'''

df_wine = pd.read_csv('../syc_data/py_machine_learning/wine.data')
df_wine.head()

我们将葡萄酒数据集划分为训练集和测试集，并使用单位方差将其标准化  

In [ ]:
'''
5
python	pandas	dataframe	values
python	sklearn	Model Selection	train_test_split()
python	sklearn	Preprocessing and Normalization	StandardScaler()
'''
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, random_state=0)
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.fit_transform(X_test)

完成数据预处理后，我们进入第二步：构造协方差矩阵  
$d\times d$维协方差矩阵，其中d位数据集的维度，此矩阵成对地存储了不同特征之间地协方差，例如两个特征$x_j和x_k$ 可通过如下公式来计算协方差：   
$$\sigma_{jk}=\frac{1}{n}\sum_{i=1}^n{(x_j^{(i)}-\mu_j)(x_k^{(i)}-\mu_k)}$$
在此，$\mu_j$和$\mu_k$分别位特征j和k的均值。注意，我们做了标准化处理后，样本的均值将为零   
两个特征之间的协方差如果位正，说明它们会同时增减，而一个负的协方差值则表示两个特征会朝相反的方向变动，一个包含三个特征的协方差矩阵可记为：5  
$$\Sigma=\left[\begin{matrix}\sigma^2_1&\sigma_{12}&\sigma_{13} \cr 
    \sigma_{21}&\sigma^2_2&\sigma_{23} \cr 
    \sigma_{31}&\sigma_{32}&\sigma^2_3 \end{matrix}\right]$$
协方差矩阵的特征向量代表主成分（最大方差方向），而对应的特征值大小就决定了特征向量的重要性  
### 问题：协方差矩阵其特征向量、特征值分别是什么？
> 特征向量和特征值是协方差矩阵通过特征分解后得到，如葡萄酒中13个特征变量——13x13协方差矩阵——13个维特征向量及13个特征值，特征向量以列的方式存储于一个13x13维的矩阵中 5  

就葡萄酒数据集来说，我们可以得到$13\times13$维协方差矩阵的13个特征向量及其对应的特征值。特征值V需满足如下条件：  
$$\Sigma{V}=\lambda{V}$$
5  
此处的特征值是$\lambda$一个标量，我们来计算协方差矩阵的特征对  
### 问题：什么是标量？
> 几何代数的概念，对应于矢量，标量亦称作无向量，用通俗的说法，标量是只有大小，没有方向的量 5  

In [ ]:
'''
python	numpy	The N-dimensional array (ndarray)	ndarray.T
python	numpy	Statistics	cov()
python	numpy	Linear algebra (numpy.linalg)	eig()
'''
import numpy as np
cov_mat = np.cov(X_train_std.T) # 转换成协方差矩阵
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat) # 将协方差矩阵进行特征分解得到特征值和特征向量
print('如下一共{:d}个特征值'.format(len(eigen_vals))) # eigen_vals特征值 eigen_vecs 特征向量
print('\nEigenvalues|特征值 \n{}'.format(eigen_vals))

### 问题：如何理解np.cov?
> 公式 $$cov(X,Y)=\frac{\Sigma^n_{i=1}{(X_i-\bar{X})(Y_i-\bar{Y})}}{n-1}$$
结果  $$C=\left[\begin{matrix}cov(1,1)&cov(1,2)&cov(1,3)&\dots&cov(1,n) \cr 
    cov(2,1)&cov(2,2)&cov(2,3)&\dots&cov(2,n) \cr 
    cov(3,1)&cov(3,2)&cov(3,3)&\dots&cov(3,n) \cr
    \dots&\dots&\dots&\dots&\dots \cr
    cov(n,1)&cov(n,2)&cov(n,3)&\dots&cov(n,n)
    \end{matrix}\right]$$

协方差矩阵计算的是不同维度之间的协方差，而不是不同样本之间。拿到一个样本矩阵，首先要明确的就是行代表什么，列代表什么，如下所示 5

In [ ]:
'''
5
python	numpy	Array manipulation routines	vstack()
'''
# 计算协方差的时候，一行代表一个特征
# 下面计算cov(T, S, M)
T = np.array([9, 15, 25, 14, 10, 18, 0, 16, 5, 19, 16, 20])
S = np.array([39, 56, 93, 61, 50, 75, 32, 85, 42, 70, 66, 80])
M = np.array([38, 56, 90, 63, 56, 77, 30, 80, 41, 79, 64, 88])
Xs = np.vstack((T, S, M))
# X每行代表一个属性
#  每列代表一个示例，或者说观测
print(np.cov(Xs))

因为要将数据集压缩到一个新的特征子空间上来实现数据降维，所以我们只选择那些包含最多信息（方差最大）的特征向量(主成分)组成子集  
由于特征值的大小决定了特征向量的重要性，因此需要将特征值按降序排列，我们感兴趣的是排序在前k个特征值所对应的特征向量。我们先绘制特征值的方差贡献率  
特征值$\lambda_j$的方差贡献率是指，特征值$\lambda_j$与所有特征值和的比值 5  
$$\frac{\lambda_j}{\sum_{j=1}^d{\lambda_j}}$$
我们可以计算并绘制出方差贡献率的累积分布图

In [ ]:
'''
python	Built-in Functions	sum()	sum()
python	Built-in Functions	sorted()	sorted() reverse
python	numpy	Mathematical functions	cumsum()
python	matplotlib	Pyplot function overview	bar() align
python	matplotlib	Pyplot function overview	step()
python	matplotlib	Pyplot function overview	legend() loc
m
'''
tot = sum(eigen_vals) # 计算所有特征值之和
var_exp = [(i/tot) for i in sorted(eigen_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
import matplotlib.pyplot as plt
plt.bar(range(1,14), var_exp, alpha=0.5, align='center',
       label='individual explained variance')
plt.step(range(1,14), cum_var_exp, where='mid',
        label='cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc='best')
plt.show()

我们应注意：PCA是一种无监督方法，意味着我们可以忽略类标信息，方差度量的是特征值在轴线上的分布 5  
## 5.1.2 特征转换
> 本节介绍了如何通过特征向量构建映射矩阵

将协方差矩阵分解为特征对（特征值和特征向量）后，我们继续执行PCA方法的最后三个步骤，将葡萄酒数据集中的信息转换到新的主成分轴上  
我们将对特征值按降序进行排列，并通过挑选出对应的特征向量构造出映射矩阵，然后使用映射矩阵将数据转换到低维的子空间上  

In [ ]:
'''
python	numpy	Mathematical functions	abs()
python	Sequence Types — list, tuple, range	sort()	sort() reverse
'''
eigen_pairs = [(np.abs(eigen_vals[i]), eigen_vecs[:, i])
               for i in range(len(eigen_vals))]
eigen_pairs.sort(reverse=True)

出于演示的需要，我们只选择两个特征值最大的特征向量，这两个值之和占据了数据集总体方差的60% 5  

In [ ]:
'''
python	numpy	Array manipulation routines	hstack()
python	numpy	Constants	newaxis
'''
w = np.hstack((eigen_pairs[0][1][:, np.newaxis],
               eigen_pairs[1][1][:, np.newaxis]))
print('Matrix W:\n{}'.format(w))

我们得到了一个13x2维的映射矩阵W。通过映射矩阵，我们可以将样本转换为一个包含两个新特征的二维向量  
$$x'=xW$$

In [ ]:
'''
python	numpy	The N-dimensional array (ndarray)	dot()
'''
X_train_pca = X_train_std.dot(w)
X_train_std[0].dot(w)

转化后的葡萄酒数据集将以124x2维矩阵的方式存储，我们以散点图的方式来对其进行可视化展示 5  

In [ ]:
'''
python	numpy	Array manipulation routines	unique()
python	numpy	Array manipulation routines	scatter() marker|c
'''
colors = ['r', 'b', 'g']
markers = ['s', 'x', 'o']
for l, c, m in zip(np.unique(y_train), colors, markers):
    plt.scatter(X_train_pca[y_train==l, 0],
                X_train_pca[y_train==l, 1],
                c=c, label=l, marker=m
                )
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend(loc='lower left')
plt.show()

相较于第二主成分（y轴），数据更多地沿着x轴（第一主成分）方向分布，线性分类器能够很好地对其进行划分 5   
## 5.1.3 使用scikit-learn进行主成分分析
> 介绍了如何使用sklearn中的PCA算法

我们使用sklearn中的PCA对葡萄酒数据集做预处理，然后使用逻辑斯蒂回归对转换后的数据进行分类  

In [ ]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

def plot_decision_regions(X, y, classifier,
                          test_idx = None, resolution = 0.02):
    # 设置散点生成器和色图
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    
    # 绘制决策边界
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                          np.arange(x2_min, x2_max, resolution)) 
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha = 0.4, cmap = cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    
    # 绘制样本分类
    X_test, y_test = X[test_idx, :], y[test_idx]
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x = X[y == cl, 0], y = X[y == cl, 1],
                   alpha = 0.8, color = cmap(idx),
                   marker = markers[idx], label = cl)

    # 高亮测试样本
    if test_idx:
        X_test, y_test = X[test_idx, :], y[test_idx]
        plt.scatter(X_test[:, 0], X_test[:, 1], c = 'yellow',
                   alpha = 0.5, linewidth = 1, marker = 'o',
                   s = 55, label = 'test set')

In [ ]:
'''
python	sklearn	Matrix Decomposition	PCA()
python	sklearn	Linear Models	LogisticRegression()
'''
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
lr = LogisticRegression()
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)
lr.fit(X_train_pca, y_train)
plot_decision_regions(X_train_pca, y_train, classifier=lr)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.legend(loc='lower left')
plt.show()

与我们自己实现的PCA相比较的话，点的分布在y轴方向上是相反的，出现这个现象的原因是特征向量可以为正或负 5  
接着我们看一下应用在转换后测试数据上的效果  

In [ ]:
plot_decision_regions(X_test_pca, y_test, classifier=lr)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.legend(loc='lower left')
plt.show()

我们通过将PCA类中n_componets参数设为None，从而可以保留所有映射后的特征，方差贡献率则可以用explained_variance_ratio_属性进行查询  

In [ ]:
'''
5
python	sklearn	Matrix Decomposition	PCA()
python	sklearn	Matrix Decomposition	PCA() explained_variance_ratio_
'''
pca = PCA(n_components=None)
X_train_pca = pca.fit_transform(X_train_std)
pca.explained_variance_ratio_

# 5.2 通过线性判别分析压缩无监督数据
> 介绍了线性判别分析(LDA)的原理、实现和使用方法

线性判别分析（LDA）也是一种特征抽取技术  
LDA和PCA都是将原始特征数据集映射到新的特征子空间，而且都是用于降低数据维度的线性转换技巧  
LDA的目标是使得映射后的特征集（往往小于原有特征空间）能够获得最佳分类效果  
LDA是监督算法，PCA是无监督算法，所以在面向分类问题上LDA要优于PCA，但A.M.Martinez提出在图像识别的某些情况下，如果每个类别只有少量样本，使用PCA更佳 5  

### 问题：什么是线性转换？
> 也叫线性变换、线性映射，是线性代数研究的一个对象，是通过加法或乘法从一个向量空间变换到另一个向量空间的过程 5  

LDA是Ronald A.Fisher于1936年针对二类别分类问题对Fisher线性判别做了最初的形式化。1948年Radhakrishna Rao通过不同类别方差相等和类别内样本呈标准正态分布的假设，将LDA泛化到了多类别分类问题上    
下图用于解释二类别分类中LDA的概念  
![5-2](../syn_pic/py_machine_learning/5-2.png)
在X轴（LD1）方向上，通过钟型曲线的波峰可以很好的将两个类分开，LD2则无法提供关于类别区分的任何信息  
**一个关于LDA的假设就是数据呈正态分布**,另外还假定各类别中数据具有相同的协方差矩阵，样本的特征从统计上来讲是相互独立的 5  
从实际应用中，即使一个或多个假设没有满足，LDA仍旧可以很好地完成降维工作  
LDA方法的关键步骤如下：
1. 对d维数据集进行标准化处理(d为特征数量)
2. 对于每一类别，计算d维的均值向量  
3. 构建类间的散布矩阵$S_B$以及类内的散布矩阵$S_W$ 5
4. 计算矩阵$S_W^{-1}S_B$的特征值及对应的特征向量  
5. 选取前k个特征值所对应的特征向量，构造一个$d\times k$维的转换矩阵W,其中特征向量以列的形式排列
6. 使用转换矩阵W将样本映射到新的特征子空间上 5  

### 问题：什么是均值向量？
> 随机向量的数学期望，就是各个特征的均值构成的向量 5  

## 5.2.1 计算散布矩阵
> 本节介绍了计算类内和类间散布矩阵的公式和方法

### 问题：什么是散布矩阵？
> 也叫散度矩阵、类内离散度矩阵或者类内离差阵，散度矩阵=协方差矩阵*(n-1)，其中n表示样本的个数，协方差矩阵可以看作是归一化的散布矩阵 5  

由于葡萄酒数据集已经做过标准化处理，我们就直接计算均值向量，均值向量$m_i$存储了类别i中样本的特征均值$\mu_m$  
$$m_i=\frac{1}{n_i}\sum^c_{x\in{D_i}}{x_m}$$
葡萄酒数据集的三个类别对应三个均值向量：  
$$m_i=\left[\begin{matrix}\mu_{i,alcohol} \cr \mu_{i,malic acid} \cr \dots \cr \mu_{i,proline} \end{matrix}\right] i\in\{1,2,3\}$$

In [ ]:
'''
5
python	numpy	Input and output	set_printoptions()
python	numpy	Statistics	mean()
'''
np.set_printoptions(precision=4)
mean_vecs = []
for label in range(1,4):
    mean_vecs.append(np.mean(
                    X_train_std[y_train==label], axis=0))
    print('MV {}: {}\n'.format(label, mean_vecs[label-1]))

通过均值向量，我们来计算类内散布矩阵$S_W$:  
$$S_W=\sum^c_{i=1}{S_i}$$
这可以通过累加各类别i的散布矩阵$S_i$来计算：  
$$S_i=\sum^c_{x\in{D_i}}{(x-m_i)(x-m_i)^T}$$

In [ ]:
'''
5
python	numpy	The N-dimensional array (ndarray)	shape

'''
d = 13 # number of features
S_W = np.zeros((d, d))
for label,mv in zip(range(1,4), mean_vecs):
    class_scatter = np.zeros((d, d))
    for row in X[y == label]:
        row, mv = row.reshape(d, 1), mv.reshape(d, 1)
        class_scatter += (row-mv).dot((row-mv).T)
    S_W += class_scatter
print('Within-class scatter matrix: {}x{}'.format(
                    S_W.shape[0], S_W.shape[1]))

我们假设训练集的类标是均匀分布的，但是，实际情况并非如此  

In [ ]:
'''
python	numpy	Statistics	bincount()
'''
print('Class label distribution:{}'.format(np.bincount(y_train)[1:]))

因此，在我们计算散布矩阵$S_W$前，需要对各类别的散布矩阵$S_i$做缩放处理  
计算散布矩阵的方式与计算协方差矩阵$\sum_i$的方式是一致的。我们用各类别单独的散布矩阵除以此类别内样本数量$N_i$  
$$\sum_i=\frac{1}{N_i}S_w=\frac{1}{N_i}\sum^c_{x\in{D_i}}{(x-m_i)(x-m_i)^T}$$
5  

In [ ]:
'''
python	numpy	Statistics	cov()
'''
d = 13 # number of features
S_W = np.zeros((d, d))
for label,mv in zip(range(1,4), mean_vecs):
    class_scatter = np.cov(X_train_std[y_train==label].T)
    S_W += class_scatter
print('Scaled Within-class scatter matrix: {}x{}'.format(
                    S_W.shape[0], S_W.shape[1]))

以上是类内散布矩阵的计算过程，随后我们来计算类间散布矩阵$S_B$  
$$S_B=\sum^c_{i=1}{N_i(m_i-m)(m_i-m)^T}$$
其中，m为全局均值，它在计算时用到了所有类别中的全部样本  

In [ ]:
'''
5
python	Built-in Functions	enumerate()	enumerate()
'''
mean_overall = np.mean(X_train_std, axis=0)
d = 13 # number of features
S_B = np.zeros((d, d))
for i, mean_vec in enumerate(mean_vecs):
    n = X[y==i+1, :].shape[0]
    mean_vec = mean_vec.reshape(d, 1)
    mean_overall = mean_overall.reshape(d, 1)
S_B += n * (mean_vec - mean_overall).dot((mean_vec - mean_overall).T)
print('Between-class scatter matrix: {}x{}'.format(
                    S_B.shape[0], S_B.shape[1]))

## 5.2.2 在新特征子空间上选取线性判别算法
> 介绍了如何通过类内和类间散布矩阵计算特征对，并利用特征值来度量类别区分能力

LDA接下来的步骤类似PCA，不过PCA是特征分解，LDA是求解$S_W^{-1}S_B$的特征对（特征向量和特征值）： 
### 问题：什么是逆矩阵？
> $S_W^{-1}$属于逆矩阵，设A是一个n阶矩阵，若存在另一个n阶矩阵B，使得： AB=BA=E ，则称方阵A可逆，并称方阵B是A的逆矩阵 5  

In [ ]:
'''
python	numpy	Linear algebra (numpy.linalg)	linalg.inv()
python	numpy	Linear algebra (numpy.linalg)	linalg.eig()
'''
eigen_vals, eigen_vecs = \
np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

随后我们按照降序对特征值进行排序：  

In [ ]:
'''
5
python	Built-in Functions	sorted()	sorted() reverse
python	Built-in Functions	sorted()	sorted() key
'''
eigen_pairs = [(np.abs(eigen_vals[i]), eigen_vecs[:,i])
               for i in range(len(eigen_vals))]
eigen_pairs = sorted(eigen_pairs,
                    key=lambda k: k[0], reverse=True)
print('Eigenvalues in decreasing order:\n')
for eigen_val in eigen_pairs:
    print(eigen_val[0])

$d\times d$维协方差矩阵的秩最大为d-1，我们只得到了一个非零特征值，实际得到的第2-13个特征值并非完全为零，是几乎等于0的实数，这是因为Numpy的精度所致  
我们达到了极少情况下的完美共线性，因为矩阵只有一个含非零特征值的特征向量，这时协方差矩阵的秩为1  
为了度量线性判别可以获取多少可区分类别的信息，我们按照特征值降序绘制出线性判别信息保持程度的图像  

In [ ]:
'''
python	numpy	The N-dimensional array (ndarray)	real
python	numpy	Mathematical functions	cumsum()
python	matplotlib	Pyplot function overview	step()
'''
tot = sum(eigen_vals.real)
discr = [(i / tot) for i in sorted(eigen_vals.real, reverse=True)]
cum_discr = np.cumsum(discr)
plt.bar(range(1, 14), discr, alpha=0.5, align='center',
       label='individual "discriminability"')
plt.step(range(1, 14), cum_discr, where='mid',
        label='cumulative "discriminability"')
plt.ylabel('"discriminability" ratio')
plt.xlabel('Linear Discriminants')
plt.ylim([-0.1, 1.1])
plt.legend(loc='best')
plt.show()

可以看到前1个线性判别几乎获取了全部有用信息 5  
下面我们来构建转换矩阵W

In [ ]:
'''
5
python	numpy	Constants	newaxis
python	numpy	The N-dimensional array (ndarray)	real
python	numpy	Array manipulation routines	hstack()
'''
w = np.hstack((eigen_pairs[0][1][:, np.newaxis].real,
              eigen_pairs[1][1][:, np.newaxis].real))
print('Matrix W:\n', w)

## 5.2.3 将样本映射到新的特征空间
> 介绍了如何通过转换矩阵W将样本映射到新的特征空间及映射后的效果  

我们可以通过乘积的方式对训练数据集进行转换：  
$$X'=XW$$

In [ ]:
'''
python	numpy	Array manipulation routines	scatter() marker|c
'''
X_train_lda = X_train_std.dot(w)
colors = ['r', 'b', 'g']
markers = ['s', 'x', 'o']
for l, c, m in zip(np.unique(y_train), colors, markers):
    plt.scatter(X_train_lda[y_train==l, 0],
               X_train_lda[y_train==l, 1],
               c=c, label=l, marker=m)
plt.xlabel('LD 1')
plt.ylabel('LD 2')
plt.legend(loc='upper right')
plt.show()

通过结果图像可见，三个葡萄酒类在特征LD1是线性可分的： 5  
## 5.2.4 使用scikit-learn进行LDA分析
> 本节介绍了如何基于sklearn使用LDA来进行降维

In [ ]:
'''
python	sklearn	Discriminant Analysis	LinearDiscriminantAnalysis
'''
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=2)
x_train_lda = lda.fit_transform(X_train_std, y_train)
x_train_lda[:5]

我们使用逻辑回归来观察在转换后数据上的表现  

In [ ]:
lr = LogisticRegression()
lr = lr.fit(X_train_lda, y_train)
plot_decision_regions(X_train_lda, y_train, classifier=lr)
plt.xlabel('LD 1')
plt.ylabel('LD 2')
plt.legend(loc='lower left')
plt.show()

下面，我们再看一下模型在测试数据集上的效果 5  

In [ ]:
'''
5
'''
X_test_lda = lda.transform(X_test_std)
plot_decision_regions(X_test_lda, y_test, classifier=lr)
plt.xlabel('LD 1')
plt.ylabel('LD 2')
plt.legend(loc='lower left')
plt.show()

# 5.3 使用核主成分分析进行非线性映射
> 本章介绍了通过核技巧PCA解决非线性问题降维的原理、实现方法、基于sklearn的应用  

在实际应用中，对于大多数情况下，我们面对非线性问题使用线性降维不是最好的方法，本章将介绍使用核PCA将数据映射到线性可分的低维空间上 5  
## 5.3.1 核函数与核技巧
> 介绍了使用基于相似(核)矩阵的技巧来实现映射的原理

第3章我们讲解SVM原理时，曾谈过用核技巧将线性不可分的特征映射到更高维线性可分的特征空间里  
定义如下的映射函数$\phi$:  
$$\phi:R^d\to R^k(k\gg d)$$
映射函数$\phi$可以将原有特征组合成新的特征，将原始的d维数据转换成更高维的k维特征空间 5  
举例来说：对于二维特征向量$x\in R^d$（这个表达式意思x是包含d个特征的列向量），采用如下映射过程转换到三维空间  
$$x=[x_1,x_2]^T$$
$$\downarrow\phi$$
$$z=[x_1^2,\sqrt{2x_1x_2},x_2^2]^T$$
### 问题：什么是列向量？
> 在线性代数中，列向量是一个 n×1 的矩阵，即矩阵由一个含有n个元素的列所组成 5  

所以核PCA的基本理念，是通过非线性映射函数将数据转换到高维空间，然后在高维空间中使用标准PCA将其映射到低维空间  
但这种方法会带来一个问题，就是这样计算时的成本非常高，所以我们需要使用核技巧来降低计算成本，核技巧的理念是在原始特征空间中计算两个高维特征空间中向量的相似度 5  
标准PCA方法中特征k和特征j之间协方差的公式如下  
$$\sigma_{jk}=\frac{1}{n}\sum_{i=1}^n{(x_j^{(i)}-\mu_j)(x_k^{(i)}-\mu_k)}$$
由于标准化处理后的均值为0，上述公式简化为：  
$$\sigma_{jk}=\frac{1}{n}\sum_{i=1}^n{x_j^{(i)}x_k^{(i)}}$$
下面是计算协方差矩阵$\sigma$的公式 5  
Bernhard Schoellkopf提出了一种方法，可以使用$\phi$通过在原始特征空间上的非线性特征组合来替代样本间点积的计算：  
$$\Sigma=\frac{1}{n}\sum_{i=1}^n{\phi(x^{(i)})\phi(x^{(i)})^T}$$
为了求得特征向量，我们求解下述公式：  
$$\Sigma{v}=\lambda{v}$$
$$\Rightarrow\frac{1}{n}\sum_{i=1}^n{\phi(x^{(i)})\phi(x^{(i)})^T}v=\lambda{v}$$
$$\Rightarrow v=\frac{1}{n\lambda}\sum_{i=1}^n{\phi(x^{(i)})[\phi(x^{(i)})^T}v]$$
5  
其中, $\lambda$和$v$分别为协方差矩阵$\Sigma$的特征值和特征向量  
注意到等式右边方括号内为标量，上式表明，当$\lambda\ne0$时，对应的特征向量v可以表示为所有$\phi(x_i)$的线性组合[1]，即  
$$v=\frac{1}{n}\sum_{i=1}^n{a^{(i)}\phi(x^{(i)})}$$
5  
这里的a可以通过提取核（相似）矩阵K的特征向量来得到  
核矩阵的推导过程如下：  
首先，使用矩阵符号来表示协方差矩阵，其中$\phi(X)$是一个$n\times k$维的矩阵：  
$$\Sigma=\frac{1}{n}\sum_{i=1}^n{\phi(x^{(i)})\phi(x^{(i)})^T}=\frac{1}{n}\phi(X)[\phi(X)]^T$$
我们可以将特征向量的公式记为： 5  
$$v=\frac{1}{n}\sum_{i=1}^n{a^{(i)}\phi(x^{(i)})}=\frac{1}{n}\phi(X)a$$
其中N维列向量 $a = [a_1,a_2,\dots,a_N]^T$  
又由于$\Sigma v=\lambda v$，我们可以得到：  
$$\frac{1}{n}\phi(X)[\phi(X)]^T\phi(X)a=\lambda\phi(X)a$$
两边都左乘矩阵$[\phi(X)]^T$，得 5  
$$\frac{1}{n}[\phi(X)]^T\phi(X)[\phi(X)]^T\phi(X)a=\lambda[\phi(X)]^T\phi(X)a$$
$$\Rightarrow\frac{1}{n}[\phi(X)]^T\phi(X)a=\lambda{a}$$
定义矩阵 $K = [\phi(X)]^T\phi(X)$，则K为$N\times N$的对称半正定矩阵，其i行j列的元素为$K_{ij}=[\phi(x_i)]^T\phi(x_j)$，将K代入得    
$$\frac{1}{n}Ka=\lambda{a}$$
K为相似（核）矩阵 5  
### 问题：什么是半正定矩阵？
> 线性代数概念  
半正定矩阵是正定矩阵的推广。实对称矩阵A称为半正定的，如果二次型$X^TAX$半正定，即对于任意不为0的实列向量X，都有$X^TAX≥0$  
正定矩阵有时会简称为正定阵。在线性代数中，正定矩阵的性质类似复数中的正实数。狭义定义：一个n阶的实对称矩阵M是正定的的条件是当且仅当对于所有的非零实系数向量z，都有$z^TMz> 0$  
5    

通过核技巧，使用核函数k以避免使用$\phi$来精确计算样本集合x中样本对之间的点积，这样我们就无需对特征向量进行精确的计算  
$$k(x^{(i)},x^{(j)})=\phi(x^{(i)})^T\phi(x^{(j)})$$
通过核PCA，我们能够得到已经映射到各成分的样本，而不像标准PCA方法那样去构建一个转换矩阵  
可以将核函数理解为：通过两个向量点积来度量向量间相似度的函数。常用的核函数有：  
多项式核：$K(x^{(i)},x^{(j)})=(x^{(i)T}x^{(j)}+\theta)^p$  5  
其中，阈值$\theta$和幂的值p需自行定义  
双曲正切(sigmoid)核：$K(x^{(i)},x^{(j)})=thah(\eta x^{(i)T}x^{(j)}+\theta)$  
径向基核函数（RBF）或者成为高斯核函数：$K(x^{(i)},x^{(j)})=exp\left(-\frac{\|x^{(i)}-x^{(j)}\|^2}{2\sigma^2}\right)$  
也可以写作:$K(x^{(i)},x^{(j)})=exp(-\gamma\|x^{(i)}-x^{(j)}\|^2)$  
综合上述讨论，我们可以通过如下三个步骤来实现一个基于RBF核的PCA: 5  
1. 为了计算核（相似）矩阵k,我们需要做如下计算：  
$$K(x^{(i)},x^{(j)})=exp(-\gamma\|x^{(i)}-x^{(j)}\|^2)$$
我们需要计算任意两样本对之间的值：  
$$K=\left[\begin{matrix}K(x^{(1)},x^{(1)})&K(x^{(1)},x^{(2)})&\dots&K(x^{(1)},x^{(n)}) \cr K(x^{(2)},x^{(1)})&K(x^{(2)},x^{(2)})&\dots&K(x^{(2)},x^{(n)}) \cr
\dots&\dots&\dots&\dots \cr 
K(x^{(n)},x^{(1)})&K(x^{(n)},x^{(2)})&\dots&K(x^{(n)},x^{(n)})\end{matrix}\right]$$
5  
2. 通过如下公式，使核矩阵k更为聚集：  
$$K'=K-1_nK-Kl_n+l_nKl_n$$
其中，$l_n$是一个$n\times n$维的矩阵（与核矩阵维度相同），其所有的值均为$\frac{1}{n}$
3. 将聚集后的核矩阵的特征值按照降序排列，选择前k个特征值所对应的特征向量。与标准PCA不同，这里的特征向量不是主成分轴，而是将样本映射到这些轴上  

为什么要在第2步中对核矩阵进行聚集处理？因为我们并没有精确计算新的特征空间，不能确定新特征空间的中心在零点 5  
## 5.3.2 使用Python实现核主成分分析  
> 介绍了如何使用Python通过三个步骤来实现KPCA  

In [ ]:
'''
5
python	scipy	Distance computations (scipy.spatial.distance)	pdist()
python	scipy	Distance computations (scipy.spatial.distance)	squareform()
python	numpy	Array creation routines	ones()
python	scipy	Linear algebra (scipy.linalg)	eigh()
python	numpy	Array manipulation routines	column_stack()
'''
from scipy.spatial.distance import pdist, squareform
from numpy import exp
from scipy.linalg import eigh
def rbf_kernel_pca(X, gamma, n_components):
    """
    RBF kernel PCA implementation
    
    Parameters
    ----------
    X: {NumPy ndarray}, shape = [n_samples, n_features]
    
    gamma: float
        Tuning parameter of the RBF kernel
    
    n_components: int
        Number of principal components to return
    
    Returns
    ---------
    X_pc: {Numpy ndarray}, shape = [n_samples, k_features]
        Projected dataset
    
    """
    # 计算成对平方欧几里得距离 Calculate pairwise squared Euclidean distances
    # in the NxN dimensional dataset.
    sq_dists = pdist(X, 'sqeuclidean')
    
    # 将成对距离转换成一个方阵 Convert pairwise distances into a square matrix.
    mat_sq_dists = squareform(sq_dists)
    
    # 计算对称核矩阵 Compute the symmetric kernel matrix
    K = exp(-gamma * mat_sq_dists)
    
    # 聚集核心矩阵 Center the kernel matrix.
    N = K.shape[0]
    one_n = np.ones((N,N)) / N
    K = K - one_n.dot(K) - K.dot(one_n) + one_n.dot(K).dot(one_n)
    
    # 从聚集的核矩阵求特征对 Obtaining eigenpairs from the centered kernel matrix 
    # numpy.eigh returns them in sorted order
    eigvals, eigvecs = eigh(K)
    
    # 收集前k个特征向量 Collect the top k eigenvectors(projected samples)
    X_pc = np.column_stack([eigvecs[:, -i]
                           for i in range(1, n_components +1)])
    return X_pc

采用RBF核函数实现的PCA进行降维时必须指定先验参数r，需要通过实验来找到一个合适的r值  
### 示例一：分离半月形数据   

### 问题：如何理解pdist?
> D = pdist(X)    计算 X 中各对行向量的相互距离(X是一个m-by-n的矩阵). 这里 D 要特别注意，D 是一个长为m(m–1)/2的行向量.可以这样理解 D 的生成：首先生成一个 X 的距离方阵，由于该方阵是对称的，令对角线上的元素为0，所以取此方阵的下三角元素，按照Matlab中矩阵的按列存储原则，此下三角各元素的索引排列即为(2,1), (3,1), ..., (m,1), (3,2), ..., (m,2), ..., (m,m–1). 5  

### 问题：如何理解squareform？
> 用来把一个向量格式的距离向量转换成一个方阵格式的距离矩阵，反之亦然。
首先输入如果是矩阵的话必须是距离矩阵，距离矩阵的特点是 
    1. d*d的对称矩阵，这里d表示点的个数
    2. 对称矩阵的主对角线都是0；
另外，如果输入的是距离向量的话，必须满足d * (d-1) / 2  
5  

### 问题：如何理解eigh?
> 求解复数矩阵-Hermitian矩阵或实对称矩阵的标准或广义特征值问题;返回正半定矩阵的特征值和特征向量 5  

我们先实现非线性示例数据集，以两个半月形状表示    

In [ ]:
'''
python	sklearn	Datasets	make_moons()
5
'''
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=100, random_state=123)
plt.scatter(X[y==0, 0], X[y==0, 1],
           color='red', marker='^', alpha=0.5)
plt.scatter(X[y==1, 0], X[y==1, 1],
           color='blue', marker='o', alpha=0.5)
plt.show()

这两个半月形是线性不可分的，我们的目标是通过核PCA将这两个半月形数据展开，首先我们尝试观察标准PCA映射后的效果   

In [ ]:
'''
python	sklearn	Matrix Decomposition	PCA()
python	matplotlib	Pyplot function overview	subplot()
python	matplotlib	axes	scatter()
python	matplotlib	axes	set_xlabel()
python	matplotlib	axes	set_ylabel()
python	matplotlib	axes	set_ylim()
python	matplotlib	axes	set_yticks()
'''
scikit_pca = PCA(n_components=2)
X_spca = scikit_pca.fit_transform(X)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(7,3))
ax[0].scatter(X_spca[y==0, 0], X_spca[y==0, 1],
             color='red', marker='^', alpha=0.5)
ax[0].scatter(X_spca[y==1, 0], X_spca[y==1, 1],
             color='blue', marker='o', alpha=0.5)

# 请注意，当我们绘制第一主成分时，为了更好地展示类间重叠
# 我们分别将三角形和圆形代表的样本向上或向下做了轻微调整（0.02）

ax[1].scatter(X_spca[y==0, 0], np.zeros((50, 1))+0.02,
             color='red', marker='^', alpha=0.5)
ax[1].scatter(X_spca[y==1, 0], np.zeros((50, 1))-0.02,
             color='blue', marker='o', alpha=0.5)
ax[0].set_xlabel('PC1')
ax[0].set_ylabel('PC2')
ax[1].set_ylim([-1,1])
ax[1].set_yticks([])
ax[1].set_xlabel('PC1')
plt.show()

请注意，PCA是无监督，过程中是未使用类标信息的，出于增强可视化效果的考虑，我们才在此使用了三角形和圆形符号用于区别类标  
现在我们将使用前一小节实现的核PCA函数:rbf_kernel_pca 5  

In [ ]:
'''
python	matplotlib	ticker	FormatStrFormatter()
python	matplotlib	axis	set_major_formatter()
'''
from matplotlib.ticker import FormatStrFormatter
X_kpca = rbf_kernel_pca(X, gamma=15, n_components=2)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(7,3))
ax[0].scatter(X_kpca[y==0, 0], X_kpca[y==0, 1],
             color='red', marker='^', alpha=0.5)
ax[0].scatter(X_kpca[y==1, 0], X_kpca[y==1, 1],
             color='blue', marker='o', alpha=0.5)
ax[1].scatter(X_kpca[y==0, 0], np.zeros((50, 1))+0.02,
             color='red', marker='^', alpha=0.5)
ax[1].scatter(X_kpca[y==1, 0], np.zeros((50, 1))-0.02,
             color='blue', marker='o', alpha=0.5)
ax[0].set_xlabel('PC1')
ax[0].set_ylabel('PC2')
ax[1].set_ylim([-1,1])
ax[1].set_yticks([])
ax[1].set_xlabel('PC1')
ax[0].xaxis.set_major_formatter(FormatStrFormatter('%0.1f'))
ax[1].xaxis.set_major_formatter(FormatStrFormatter('%0.1f'))
plt.show()

可以看到，两个类别此时是线性可分的，这使得转换后的数据适合作为线性分类器的训练数据集  
不过，对于可调整参数$\gamma$，没有一个通用的值使其适用于不同的数据集  
### 示例二：分离同心圆
我们再看一下另外一个关于非线性问题的例子-同心圆  

In [ ]:
'''
5
python	sklearn	Datasets	make_circles()
'''
from sklearn.datasets import make_circles
X, y = make_circles(n_samples=1000,
                   random_state=123, noise=0.1, factor=0.2)
plt.scatter(X[y==0, 0], X[y==0, 1],
           color='red', marker='^', alpha=0.5)
plt.scatter(X[y==1, 0], X[y==1, 1],
           color='blue', marker='^', alpha=0.5)
plt.show()